<a href="https://colab.research.google.com/github/illNO/web_dev/blob/master/Bert_ag_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 194kB 21.3MB/s 
     |████████████████████████████████| 245kB 15.6MB/s 
     |████████████████████████████████| 112kB 55.3MB/s 
     |████████████████████████████████| 2.0MB 14.3MB/s 
     |████████████████████████████████| 3.2MB 49.5MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c4a66f7770be56c5657c2774fc11573736158d9e5c44c2b2124850240f5aed2d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
# import nlp
import datasets
from datasets import load_dataset
import spacy
import sklearn
import numpy as np
import transformers
import torch
from torch import nn

In [16]:
bert = transformers.AutoModel.from_pretrained("bert-base-uncased")

In [17]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
tokenizer.tokenize("My name is Igor Shevchuk")

['my', 'name', 'is', 'igor', 'she', '##vc', '##hu', '##k']

In [33]:
transformers.AutoModelForSequenceClassification.from_pretrained?

In [6]:
bert_classification = transformers.AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
bert_classification

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [8]:
train_data

In [8]:
def tokenize(batch):
  return tokenizer(batch['text'], padding=True, truncation=True)

train_data, test_data = datasets.load_dataset('ag_news', split=['train', 'test'])
# train_data = train_data[:60000]

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data = test_data.map(tokenize, batched=True, batch_size=len(train_data))

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a. Subsequent calls will reuse this data.


In [13]:
from sklearn.metrics import accuracy_score
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc
  }

In [14]:
training_args = transformers.TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-4,
    warmup_steps=150,
    logging_dir='./logs',
    evaluation_strategy='epoch'
)
trainer = transformers.Trainer(
    model = bert_classification,
    args = training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data, 
    eval_dataset=test_data
)
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.217300,0.228138,0.932237,68.772900,110.509000
2,0.122900,0.199875,0.940000,68.816500,110.439000


TrainOutput(global_step=7500, training_loss=0.18845680236816406, metrics={'train_runtime': 8727.0543, 'train_samples_per_second': 0.859, 'total_flos': 3.654220563648e+16, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 2947558, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 96703821, 'init_mem_gpu_peaked_delta': 537184768, 'train_mem_cpu_alloc_delta': 561340, 'train_mem_gpu_alloc_delta': 536897536, 'train_mem_cpu_peaked_delta': 99045121, 'train_mem_gpu_peaked_delta': 8189415424})